# PROCESO DE ETL 

## CONSIDERACIONES GENERALES

Se desea preparar un conjunto de datos "limpio" para iniciar el proceso de análisis exploratorio. Como consideración fundamental, se deben conservar la cantidad de registros originales durante las transformaciones. Es decir, no se deben eliminar registros debido a la falta de información sus campos (a exepción de duplicados). El objetivo principal es contar la totalidad de eventos ocurridos.



## Carga de datos

In [82]:
import pandas as pd
import numpy as np
import os

# Directorio donde se encuentra el archivo Excel
directorio = '_src/data/'
# Nombre del archivo Excel
archivo_excel = 'homicidios.xlsx'
# Ruta completa al archivo Excel
ruta_excel = os.path.join(directorio, archivo_excel)

# Leer la solapa 'HECHOS' del archivo Excel
datos_hechos = pd.read_excel(ruta_excel, sheet_name='HECHOS')

# Leer la solapa 'VICTIMAS' del archivo Excel
datos_victimas = pd.read_excel(ruta_excel, sheet_name='VICTIMAS')


## Dataset 'Hechos'

Se comienza el proceso de limpieza sobre esta base de datos.

In [83]:
datos_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

## Eliminar datos 

Se eliminan aquellas columnas que son redundantes según mi análisis, o que no serán tenidas en cuenta para la exploración de datos. 

In [84]:
# Lista de columnas a eliminar
columnas_a_eliminar = ['HORA', 'LUGAR_DEL_HECHO', 'Altura', 'Cruce', 'XY (CABA)','PARTICIPANTES']

# Eliminar las columnas especificadas
datos_hechos = datos_hechos.drop(columnas_a_eliminar, axis=1)

## Verificar tipo de datos

Se convirten los diferentes campos al tipo de datos correspondiente y se reemplazan los datos nulos por valores NaN. 

In [85]:
# Convertir todas las columnas especificadas a tipo numérico
columnas_numericas = ['N_VICTIMAS', 'AAAA', 'MM', 'DD', 'HH', 'pos x', 'pos y', 'COMUNA']
datos_hechos[columnas_numericas] = datos_hechos[columnas_numericas].apply(pd.to_numeric, errors='coerce')

# Contar los valores NaN en las columnas convertidas
valores_nulos = datos_hechos[columnas_numericas].isnull().sum()

# Mostrar los valores NaN encontrados en cada columna
print("Valores NaN en las columnas convertidas:")
print(valores_nulos)

Valores NaN en las columnas convertidas:
N_VICTIMAS     0
AAAA           0
MM             0
DD             0
HH             1
pos x         12
pos y         12
COMUNA         0
dtype: int64


In [86]:
datos_hechos.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HH,TIPO_DE_CALLE,Calle,Dirección Normalizada,COMUNA,pos x,pos y,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,4.0,AVENIDA,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,-58.475340,-34.687570,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,1.0,GRAL PAZ,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,-58.508775,-34.669777,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,7.0,AVENIDA,ENTRE RIOS AV.,ENTRE RIOS AV. 2034,1,-58.390403,-34.631894,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,0.0,AVENIDA,LARRAZABAL AV.,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,-58.465039,-34.680930,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,5.0,AVENIDA,SAN JUAN AV.,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,-58.387183,-34.622466,MOTO,PASAJEROS


In [87]:
# Verificar si los datos en la columna 'FECHA' son todos en formato de fecha
datos_hechos['FECHA'] = pd.to_datetime(datos_hechos['FECHA'], errors='coerce')

# Contar los valores NaN en las columnas convertidas
valores_nulos = datos_hechos['FECHA'].isnull().sum()

# Mostrar los valores NaN encontrados en cada columna
print("Valores NaN en las columnas convertidas:")
print(valores_nulos)

Valores NaN en las columnas convertidas:
0


In [88]:
# Reemplazar los registros vacíos por NaN en las columnas 'calle' y 'direccion normalizada'
datos_hechos['Calle'] = datos_hechos['Calle'].replace('', np.nan)
datos_hechos['Dirección Normalizada'] = datos_hechos['Dirección Normalizada'].replace('', np.nan)


In [89]:
# Reemplazar los valores 'SD' por NaN en las columnas 'VICTIMA' y 'ACUSADO'
datos_hechos['VICTIMA'] = datos_hechos['VICTIMA'].replace('SD', np.nan)
datos_hechos['ACUSADO'] = datos_hechos['ACUSADO'].replace('SD', np.nan)


In [90]:
# Reemplazar los valores 0 por NaN en la columna 'COMUNA'
datos_hechos['COMUNA'] = datos_hechos['COMUNA'].replace(0, np.nan)

## Agregado de nombre a la variable categórica COMUNA

Fuente: https://www.argentina.gob.ar/caba/comunas

In [91]:
comunas_dict = {
    1: 'Retiro, San Nicolás, Puerto Madero, San Telmo, Monserrat, Constitución',
    2: 'Recoleta',
    3: 'San Cristóbal, Balvanera',
    4: 'Barracas, La Boca',
    5: 'Almagro, Boedo',
    6: 'Caballito',
    7: 'Flores, Parque Chacabuco',
    8: 'Villa Soldati, Villa Riachuelo, Villa Lugano',
    9: 'Liniers, Mataderos, Parque Avellaneda',
    10: 'Villa Real, Monte Castro, Versalles, Floresta, Vélez Sársfield, Villa Luro',
    11: 'Villa General Mitre, Villa Devoto, Villa del Parque, Villa Santa Rita',
    12: 'Coghlan, Saavedra, Villa Urquiza, Villa Pueyrredón',
    13: 'Núñez, Belgrano, Colegiales',
    14: 'Palermo',
    15: 'Chacarita, Villa Crespo, La Paternal, Villa Ortúzar, Agronomía, Parque Chas'
}

datos_hechos['COMUNA_NOMBRE'] = datos_hechos['COMUNA'].map(comunas_dict)


## Agregado de población por comuna

Se agregar el valor de población por comuna de CABA, que se utilizará para el cálculo deL KPI solicitado, y además para el análisis de correlación con otras variables.

Se toma información del censo 2022, y se establece igual valor de población para todos los años de estudio.


Fuente: https://es.wikipedia.org/wiki/Anexo:Comunas_de_la_ciudad_de_Buenos_Aires_por_poblaci%C3%B3n

In [92]:
comunas_dict = {
    1: 223282,
    2: 158368,
    3: 236294,
    4: 230945,
    5: 193859,
    6: 203784,
    7: 216832,
    8: 204842,
    9: 169538,
    10: 171797,
    11: 203491,
    12: 236294,
    13: 265199,
    14: 249016,
    15: 197903
}

datos_hechos['COMUNA_POB'] = datos_hechos['COMUNA'].map(comunas_dict)

## Agregado día de semana

Se incorpora al dataset la identificación del día de la semana correspondiente a la fecha del accidente.

In [93]:
# Crear una nueva columna con el día de la semana en valor numérico (0 para lunes, 1 para martes, etc.)
datos_hechos['DIA_SEMANA_NUM'] = datos_hechos['FECHA'].dt.dayofweek



In [94]:
# Diccionario para mapear los números de los días de la semana a los nombres en español
dias_semana_dict = {
    0: 'Lunes',
    1: 'Martes',
    2: 'Miércoles',
    3: 'Jueves',
    4: 'Viernes',
    5: 'Sábado',
    6: 'Domingo'
}

# Crear una nueva columna con el día de la semana en valor de cadena
datos_hechos['DIA_SEMANA_STR'] = datos_hechos['DIA_SEMANA_NUM'].map(dias_semana_dict)

# Mostrar el DataFrame resultante
print(datos_hechos)


            ID  N_VICTIMAS      FECHA  AAAA  MM  DD    HH TIPO_DE_CALLE  \
0    2016-0001           1 2016-01-01  2016   1   1   4.0       AVENIDA   
1    2016-0002           1 2016-01-02  2016   1   2   1.0      GRAL PAZ   
2    2016-0003           1 2016-01-03  2016   1   3   7.0       AVENIDA   
3    2016-0004           1 2016-01-10  2016   1  10   0.0       AVENIDA   
4    2016-0005           1 2016-01-21  2016   1  21   5.0       AVENIDA   
..         ...         ...        ...   ...  ..  ..   ...           ...   
691  2021-0093           1 2021-12-13  2021  12  13  17.0       AVENIDA   
692  2021-0094           1 2021-12-20  2021  12  20   1.0     AUTOPISTA   
693  2021-0095           1 2021-12-30  2021  12  30   0.0       AVENIDA   
694  2021-0096           1 2021-12-15  2021  12  15  10.0       AVENIDA   
695  2021-0097           1 2021-11-18  2021  11  18   6.0         CALLE   

                            Calle  \
0                PIEDRA BUENA AV.   
1                  PAZ, G

## Dataset 'Victimas'

Se realizan los mismos procesos para esta base de datos. 

In [95]:
datos_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [96]:
# Lista de columnas a eliminar
columnas_a_eliminar = ['FECHA', 'AAAA', 'MM', 'DD', 'VICTIMA']

# Eliminar las columnas especificadas
datos_victimas = datos_victimas.drop(columnas_a_eliminar, axis=1)

In [97]:
# Convertir todas las columnas especificadas a tipo numérico
columnas_numericas = ['EDAD']
datos_victimas[columnas_numericas] = datos_victimas[columnas_numericas].apply(pd.to_numeric, errors='coerce')

# Contar los valores NaN en las columnas convertidas
valores_nulos = datos_victimas[columnas_numericas].isnull().sum()

# Mostrar los valores NaN encontrados en cada columna
print("Valores NaN en las columnas convertidas:")
print(valores_nulos)

Valores NaN en las columnas convertidas:
EDAD    53
dtype: int64


In [98]:
# Verificar si los datos en la columna 'FECHA' son todos en formato de fecha
datos_victimas['FECHA_FALLECIMIENTO'] = pd.to_datetime(datos_victimas['FECHA_FALLECIMIENTO'], errors='coerce')

# Contar los valores NaN en las columnas convertidas
valores_nulos = datos_victimas['FECHA_FALLECIMIENTO'].isnull().sum()

# Mostrar los valores NaN encontrados en cada columna
print("Valores NaN en las columnas convertidas:")
print(valores_nulos)

Valores NaN en las columnas convertidas:
68


In [99]:
datos_victimas.head()

,ID_hecho,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,CONDUCTOR,MASCULINO,19.0,2016-01-01
1,2016-0002,CONDUCTOR,MASCULINO,70.0,2016-01-02
2,2016-0003,CONDUCTOR,MASCULINO,30.0,2016-01-03
3,2016-0004,CONDUCTOR,MASCULINO,18.0,NaT
4,2016-0005,CONDUCTOR,MASCULINO,29.0,2016-02-01


In [100]:
# Reemplazar los valores 'SD' por NaN en las columnas 'VICTIMA' y 'ACUSADO'
datos_victimas['ROL'] = datos_victimas['ROL'].replace('SD', np.nan)
datos_victimas['SEXO'] = datos_victimas['SEXO'].replace('SD', np.nan)

In [101]:
# Guardo los archivos por separado para trabajarlos en el dashboard

datos_hechos.to_csv('_src/data/datos_hechos_v2.csv', index=False)
datos_victimas.to_csv('_src/data/datos_victimas_v2.csv', index=False)


## Union de datasets

In [102]:
# Unir los DataFrames por las columnas ID y ID_hecho
datos_completos = datos_hechos.merge(datos_victimas, left_on='ID', right_on='ID_hecho', how='left')

datos_completos = datos_completos.drop('ID', axis=1)

In [103]:
# Renombrar las columnas 
datos_completos = datos_completos.rename(columns={'ID_hecho': 'ID_HECHO', 'Calle': 'CALLE', 'Dirección Normalizada': 'DIRE_NOR', 'pos x': 'POSX', 'pos y': 'POSY'})


In [104]:
datos_completos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   N_VICTIMAS           717 non-null    int64         
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   HH                   716 non-null    float64       
 6   TIPO_DE_CALLE        717 non-null    object        
 7   CALLE                716 non-null    object        
 8   DIRE_NOR             708 non-null    object        
 9   COMUNA               715 non-null    float64       
 10  POSX                 704 non-null    float64       
 11  POSY                 704 non-null    float64       
 12  VICTIMA              708 non-null    object        
 13  ACUSADO              694 non-null  

In [105]:
# Ordenar las columnas
columnas_ordenadas = ['ID_HECHO', 'FECHA', 'AAAA', 'MM', 'DD', 'HH', 'DIA_SEMANA_NUM', 'DIA_SEMANA_STR', 'N_VICTIMAS', 'ACUSADO', 'VICTIMA', 'ROL', 'SEXO', 'EDAD', 'FECHA_FALLECIMIENTO', 'COMUNA', 'COMUNA_NOMBRE','COMUNA_POB','TIPO_DE_CALLE', 'CALLE', 'DIRE_NOR', 'POSX', 'POSY']

# Seleccionar y ordenar las columnas en el DataFrame
datos_completos = datos_completos[columnas_ordenadas]

In [106]:
datos_completos.head()

,ID_HECHO,FECHA,AAAA,MM,DD,HH,DIA_SEMANA_NUM,DIA_SEMANA_STR,N_VICTIMAS,ACUSADO,...,EDAD,FECHA_FALLECIMIENTO,COMUNA,COMUNA_NOMBRE,COMUNA_POB,TIPO_DE_CALLE,CALLE,DIRE_NOR,POSX,POSY
0,2016-0001,2016-01-01,2016,1,1,4.0,4,Viernes,1,AUTO,...,19.0,2016-01-01,8.0,"Villa Soldati, Villa Riachuelo, Villa Lugano",204842.0,AVENIDA,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",-58.475340,-34.687570
1,2016-0002,2016-01-02,2016,1,2,1.0,5,Sábado,1,PASAJEROS,...,70.0,2016-01-02,9.0,"Liniers, Mataderos, Parque Avellaneda",169538.0,GRAL PAZ,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",-58.508775,-34.669777
2,2016-0003,2016-01-03,2016,1,3,7.0,6,Domingo,1,AUTO,...,30.0,2016-01-03,1.0,"Retiro, San Nicolás, Puerto Madero, San Telmo,...",223282.0,AVENIDA,ENTRE RIOS AV.,ENTRE RIOS AV. 2034,-58.390403,-34.631894
3,2016-0004,2016-01-10,2016,1,10,0.0,6,Domingo,1,NaN,...,18.0,NaT,8.0,"Villa Soldati, Villa Riachuelo, Villa Lugano",204842.0,AVENIDA,LARRAZABAL AV.,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",-58.465039,-34.680930
4,2016-0005,2016-01-21,2016,1,21,5.0,3,Jueves,1,PASAJEROS,...,29.0,2016-02-01,1.0,"Retiro, San Nicolás, Puerto Madero, San Telmo,...",223282.0,AVENIDA,SAN JUAN AV.,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",-58.387183,-34.622466


In [107]:
# Guardar el DataFrame en un archivo CSV
datos_completos.to_csv('_src/data/datos_completos.csv', index=False)
